<a href="https://colab.research.google.com/github/Braga19/ClipSoundSymb/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MAIN

In [ ]:
import pandas as pd

##IMAGE GENERATION

In [ ]:
%%capture
!pip install diffusers
!pip install transformers
!pip install sentence-transformers
!git clone https://github.com/vinid/social-diffusers
!cd social-diffusers ; git pull
!cd social-diffusers ; pip install -e .

In [ ]:
from social_diffusers import diffs
import numpy as np
from torch import autocast
from diffusers import StableDiffusionPipeline
import pathlib

In [ ]:
#import stable diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
	"CompVis/stable-diffusion-v1-4", 
	use_auth_token=""
).to("cuda")

In [ ]:
#importing wrapper for generating multiple images at the same time (credits to Federico Bianchi)
diffs_gen = diffs.Diffs(hf_token="", model_type="CompVis/stable-diffusion-v1-4")
diffs_gen.pipe.safety_checker = lambda images, clip_input: (images, False) # disable NSFW classifier

In [ ]:
#upload datset of fictional characters
df = read.read_df('/content/drive/MyDrive/Thesis2023/dataset/avgRatings_annotated.csv')
#extracting unique names
characters = df.name.unique().tolist()
characters = [k.lower() for k in characters]

In [ ]:
#generating 100 images per name and storing in drive
prompts = ["a face", "a drawing"]

for character in characters:
  
  for prompt in prompts:

    for k in range(50):

      query = f'{prompt} of a character called {character}'

      _, image = diffs_gen.generate_image_embedding(query, num_images=1, return_image=True)

      p = pathlib.Path(f"drive/MyDrive/Character_images_firstgen/{character}/{prompt}")
      p.mkdir(parents=True, exist_ok=True)

      image[0][0].save(f"drive/MyDrive/Character_images_firstgen/{character}/{prompt}/{character}_{k}.jpeg")

##IMAGE CLASSIFICATION

In [1]:
!pip install torch
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.4 MB/s eta 0:00:00


In [1]:
%cd /content/drive/MyDrive/Thesis2023

/content/drive/MyDrive/Thesis2023


In [2]:
from src import read, classification, adjust, gender, emotions, age
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

In [ ]:
#!cp -r /content/Thesis2023 /content/drive/MyDrive/Thesis2023

In [3]:
df = read.read_df('/content/drive/MyDrive/Thesis2023/dataset/avgRatings_annotated.csv')
root_dir = '/content/drive/MyDrive/Character_images_firstgen'
read.mount()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Gender Classification

In [ ]:
#subset df for names with ratings for gender(-50 male, 50 female)
gender_df = adjust.subset(df, 'name', 'rating.mean','attribute','gender')
gender_df['name'] = gender_df['name'].str.lower()




<ipython-input-4-e8ab7569f489>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_df_5['name'] = gender_df_5['name'].str.lower()


In [4]:
#importing classifier rizvan, rizvan2 and leilab
extractor_rizvan = AutoFeatureExtractor.from_pretrained("rizvandwiki/gender-classification")
model_rizvan = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification")

extractor_rizvan2 = AutoFeatureExtractor.from_pretrained("rizvandwiki/gender-classification-2")
model_rizvan2 = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification-2")

extractor_leilab = AutoFeatureExtractor.from_pretrained("Leilab/gender_class")
model_leilab = AutoModelForImageClassification.from_pretrained("Leilab/gender_class")


/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
#Classifcation
result = classification.gender(model, extractor, gender_df, root_dir)
gender_prediction = result.to_csv('gender_prediction.csv', index=False)
!cp gender_prediction.csv /content/drive/MyDrive/Thesis2023/dataset

Some useful code for averaging probability scores and merging df

```
predict_df.groupby(['name', 'prompt'])['female_prob'].mean().reset_index()
predict_df_avg = predict_df.groupby('name')['female_prob'].mean().reset_index()
merged_df = pd.merge(predict_df_avg, gender_df_5[['name', 'rating.scale']], on="name")

```



### Emotion Classification

In [ ]:
#subsetting df with names with polarity ratings
emotion_df = adjust.subset(df, 'name', 'rating.mean','attribute', 'polarity')
emotion_df['name'] = emotion_df['name'].str.lower()

#Classifier emotions 
extractor1 = AutoFeatureExtractor.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")
model1 = AutoModelForImageClassification.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")

#Emotions classifcation
result_emotion = classification.emotion(model1, extractor1, emotion_df, root_dir)
emotion_prediction = result_emotion.to_csv('emotion_prediction.csv', index=False)
!cp emotion_prediction.csv /content/drive/MyDrive/Thesis2023/dataset


### Age Classification

In [ ]:
#subsetting names with ratings for age
age_df = adjust.subset(df, 'name', 'rating.mean','attribute', 'age')
age_df['name'] = age_df['name'].str.lower()

#classifier age
extractor_age = AutoFeatureExtractor.from_pretrained("nateraw/vit-age-classifier")
model_age = AutoModelForImageClassification.from_pretrained("nateraw/vit-age-classifier")


<ipython-input-10-5eb1fcd3054b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df_5['name'] = age_df_5['name'].str.lower()


/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
result_age = classification.age(model_age, extractor_age, age_df_5, root_dir)
age_prediction = result_age.to_csv('age_prediction.csv', index=False)
!cp age_prediction.csv /content/drive/MyDrive/Thesis2023/dataset

NameError: ignored

### Trial with toy dataset

Gender

In [4]:
#subsetting five elements for trial gender
gender_df = adjust.subset(df, 'name', 'rating.mean','attribute','gender')
gender_df_sub = gender_df[0:3]
gender_df_sub['name'] = gender_df_sub['name'].str.lower()

<ipython-input-4-545bde9ba6ba>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_df_sub['name'] = gender_df_sub['name'].str.lower()


In [5]:
#importing classifier rizvan, rizvan2 and leilab
extractor_rizvan = AutoFeatureExtractor.from_pretrained("rizvandwiki/gender-classification")
model_rizvan = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification")

extractor_rizvan2 = AutoFeatureExtractor.from_pretrained("rizvandwiki/gender-classification-2")
model_rizvan2 = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification-2")

extractor_leilab = AutoFeatureExtractor.from_pretrained("Leilab/gender_class")
model_leilab = AutoModelForImageClassification.from_pretrained("Leilab/gender_class")

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [6]:
#classification rizvan1, rizvan2, leilab
result_rizvan = gender.rizvan(model_rizvan, extractor_rizvan, gender_df_sub, root_dir)
result_rizvan2 = gender.rizvan(model_rizvan2, extractor_rizvan2, gender_df_sub, root_dir)
result_leilab = gender.leilab(model_leilab, extractor_leilab, gender_df_sub, root_dir)

Emotions

In [7]:
#subsetting five elements for trial emotions
emotion_df = adjust.subset(df, 'name', 'rating.mean','attribute','polarity')
emotion_df_sub = emotion_df[0:3]
emotion_df_sub['name'] = emotion_df_sub['name'].str.lower()

<ipython-input-7-23fa271af7e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emotion_df_sub['name'] = emotion_df_sub['name'].str.lower()


In [8]:
#importing classifier kdh, jaygoogle, jayres
extractor_kdh = AutoFeatureExtractor.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")
model_kdh = AutoModelForImageClassification.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")

extractor_jaygoogle = AutoFeatureExtractor.from_pretrained("jayanta/google-vit-base-patch16-224-cartoon-emotion-detection")
model_jaygoogle = AutoModelForImageClassification.from_pretrained("jayanta/google-vit-base-patch16-224-cartoon-emotion-detection")

extractor_jayres = AutoFeatureExtractor.from_pretrained("jayanta/microsoft-resnet-50-cartoon-emotion-detection")
model_jayres = AutoModelForImageClassification.from_pretrained("jayanta/microsoft-resnet-50-cartoon-emotion-detection")

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


/usr/local/lib/python3.9/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [9]:
result_kdh = emotions.kdh(model_kdh, extractor_kdh, emotion_df_sub, root_dir)
result_jaygoogle = emotions.jayanta(model_jaygoogle, extractor_jaygoogle, emotion_df_sub, root_dir)
result_jayres = emotions.jayanta(model_jayres, extractor_jayres, emotion_df_sub, root_dir)

Age

In [10]:
age_df = adjust.subset(df, 'name', 'rating.mean','attribute','age')
age_df_sub = age_df[0:3]
age_df_sub['name'] = age_df_sub['name'].str.lower()

<ipython-input-10-3a9e887587b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df_sub['name'] = age_df_sub['name'].str.lower()


In [11]:
#importing classifier nateraw, ibombVit, ibombSwin

extractor_nateraw = AutoFeatureExtractor.from_pretrained("nateraw/vit-age-classifier")
model_nateraw = AutoModelForImageClassification.from_pretrained("nateraw/vit-age-classifier")

extractor_ibombVit = AutoFeatureExtractor.from_pretrained("ibombonato/vit-age-classifier")
model_ibombVit = AutoModelForImageClassification.from_pretrained("ibombonato/vit-age-classifier")

extractor_ibombSwin = AutoFeatureExtractor.from_pretrained("ibombonato/swin-age-classifier")
model_ibombSwin = AutoModelForImageClassification.from_pretrained("ibombonato/swin-age-classifier")

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [12]:
result_nateraw = age.nateraw(model_nateraw, extractor_nateraw, age_df_sub, root_dir)
result_ibombVit = age.ibomb(model_ibombVit, extractor_ibombVit, age_df_sub, root_dir)
result_ibombSwin = age.ibomb(model_ibombSwin, extractor_ibombSwin, age_df_sub, root_dir)



In [13]:
result_nateraw = age.nateraw(model_nateraw, extractor_nateraw, age_df_sub, root_dir)


In [18]:
classifier = [result_rizvan, result_rizvan2, result_leilab, result_kdh, result_jaygoogle, result_jayres, result_nateraw, result_ibombVit, result_ibombSwin]
names = ['rizvan', 'rizvan2', 'leilab', 'kdh', 'jaygoogle', 'jayres', 'nateraw','ibombVit', 'ibombSwin']
for nr,x in enumerate(classifier):
  x.to_csv(f'prediction_{names[nr]}.csv', index=False)
  

